**FUNCIÓN ULTIMATE**

In [1]:
#Importación de librerias
import pandas as pd 
import numpy as np
import re

**DECLARACIÓN DE FUNCIÓN**

In [2]:
def limpiar_correos(input_csv, output_csv, correos_eliminados_csv, wrong_words_regex):
    """
    Elimina correos electrónicos que contienen patrones no deseados (basados en regex) de un archivo CSV
    y elimina duplicados en el campo 'EMAIL' cuya frecuencia sea mayor a 4.

    Parámetros:
    input_csv (str): Ruta del archivo CSV de entrada.
    output_csv (str): Ruta del archivo CSV de salida.
    correos_eliminados_csv (str): Ruta del archivo CSV donde se guardarán los correos eliminados.
    wrong_words_regex (list): Lista de expresiones regulares con palabras o patrones a eliminar.

    Retorna:
    correos_eliminados (list): Lista de correos electrónicos eliminados.
    """
    # Cargar el archivo 
    df = pd.read_csv(input_csv, on_bad_lines = 'skip', low_memory=False,)

    # Crear una máscara booleana para seleccionar filas donde 'EMAIL' no es nulo y filtrar
    print("SECCIÓN DE NULOS")
    print("Número de registros antes de eliminar valores nulos en 'EMAIL':", len(df))
    df = df[df['EMAIL'].notna()].reset_index(drop=True)

    # Imprimir el resultado para verificar los NA
    print("Número de registros después de eliminar valores nulos en 'EMAIL':", len(df))
    print()

    # Limpiar la columna 'EMAIL' eliminando espacios al inicio-final como intermedios y convirtiendo a minúsculas
    df['EMAIL'] = df['EMAIL'].apply(lambda email: str(email).lower().strip().replace(' ',''))

    # Crear listas vacías para almacenar los índices y correos a eliminar
    indices_a_eliminar = []
    correos_eliminados = []

       # Iterar sobre cada correo electrónico en la columna 'EMAIL'
    for index, email in df['EMAIL'].items():
        # Verificar si el correo coincide con algún patrón de la lista regex
        if any(re.search(pattern, email) for pattern in wrong_words_regex):
            # Agregar el índice y correo a las listas si coincide
            indices_a_eliminar.append(index)
            correos_eliminados.append(email)

    # Eliminar las filas con los correos electrónicos no deseados
    df_clean = df.drop(indices_a_eliminar)

    # Contar la frecuencia de cada correo en el campo 'EMAIL'
    frecuencia_correos = df_clean['EMAIL'].value_counts()

    # Obtener los correos que aparecen más de 4 veces
    #Este Dataframe no se esta exportando en Correos_eliminados para distinguir los que son eliminado por regex y por duplicados.
    
    correos_repetidos = frecuencia_correos[frecuencia_correos > 4].index

    # Contar el número de registros que tienen una frecuencia mayor a 4
    registros_frecuencia_mayor_a_4 = len(correos_repetidos)
    print("SECCIÓN DE REGISTROS CON FRECUENCIAS ALTAS")
    print(f"Número de correos con frecuencia mayor a 4: {registros_frecuencia_mayor_a_4}")


    # Contar el número de filas antes de la eliminación de duplicados
    filas_antes = len(df_clean)

    # Filtrar el DataFrame para conservar solo las filas que no contienen estos correos duplicados
    df_clean = df_clean[~df_clean['EMAIL'].isin(correos_repetidos)]
    df_clean.reset_index(drop= True, inplace= True)

    # Contar el número de filas después de la eliminación
    filas_despues = len(df_clean)

    # Calcular el número de filas eliminadas
    filas_eliminadas = filas_antes - filas_despues
    print(f"Número de filas eliminadas debido a duplicados con frecuencia mayor a 4: {filas_eliminadas}")
    print()

    # Rellenar valores nulos y limpiar caracteres no alfabéticos directamente
    df_clean['NOMBRES'] = df_clean['NOMBRES'].str.strip()  # Elimina espacios al inicio y al final
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace('', np.nan)  # Reemplaza cadenas vacías con NaN
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace(' ', np.nan) #Remplzar los espacios por estimado
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace('  ', np.nan) #Remplzar los espacios por estimado
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace('   ', np.nan) #Remplzar los espacios por estimado
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace(r'[^a-zA-Z\s]', np.nan, regex=True) #elimina caracteres no alfabéticos
    df_clean['NOMBRES'] = df_clean['NOMBRES'].fillna('ESTIMADO')  # Llena valores nulos con 'ESTIMADO'
    df_clean['NOMBRES'] = df_clean['NOMBRES'].replace('MA', 'MARIA') #Remplazar los MA con María 

    
    
    # Contar cuántos nombres se han llenado con 'ESTIMADO'
    print("SECCIÓN DE LLENADO DE NOMBRES VACÍOS")
    print("Cantidad de nombres llenados con 'ESTIMADO':", (df_clean['NOMBRES'] == 'ESTIMADO').sum())
    print()

    # Iterar sobre cada valor en la columna 'NAMES' para quitar puntos
    for index, name in df_clean['NOMBRES'].items():
        # Reemplazar los puntos en el valor de 'NAMES'
        df_clean.at[index, 'NOMBRES'] = name.replace('.', '')
    

    # Mostrar el número de correos eliminados
    print("SECCIÓN DE CORREOS ELIMINADOS POR REGEX")
    print(f"Número de correos eliminados por patrones no deseados: {len(correos_eliminados)}")
    print()

    # Contabilizar el número de REPEP
    print('SECCIÓN DE REPEP')
    print(f'El número de registros en el DataFrame antes de eliminar REPEP:{len(df_clean)}')
    repep = len(df_clean[df_clean['FLG_REPEP'] == 'Y'])
    print(f'El número de registros REPEP es de : {repep}')
    #Eliminación de registros REPEP
    df_clean = df_clean[df_clean['FLG_REPEP']!='Y'] 
    print()
    # Verificar la eliminación
    print(f'El número de registros en el DataFrame después de eliminar REPEP: {len(df_clean)}')
    
    
    #Mostrar el número de registros totales en el DF limpio
    num_registros = len(df_clean)
    print(f'El número de registros en el DataFrame despúes de la limpieza es de: {num_registros}')

    # Guardar la lista de correos eliminados en un archivo CSV
    pd.DataFrame(correos_eliminados, columns=['Correo Eliminado']).to_csv(correos_eliminados_csv, index=False)

    # Guardar el DataFrame limpio en un nuevo archivo CSV
    df_clean.to_csv(output_csv, index=False)

    # Retornar la lista de correos eliminados
    return 'Felicidades, has limpiado tu base con éxito'

    


**LISTA REGEX**

In [3]:
wrong_words_regex = [
    r'@@',                               # Doble arroba
    r'\.\.',                             # Doble punto
    r'\.com@',                           # ".com" seguido de arroba
    r'gm?ial|gnial|dumy|notene',         # Errores tipográficos comunes,
    r'@(yopmail)',                       # Excluir solo el dominio "yopmail"
    r'(dummy|prueba|ejemplo|temporal)',  # Palabras genéricas o placeholders
    r'[\+\&:,"\' ]',                     # Caracteres especiales o espacios
    r'\bno_tiene\b|\bNO TIENE\b',        # Frases exactas (en mayúsculas y minúsculas)
    r'\b(no|sin)_correo\b',              # "no correo" o "sin correo"
    r'\b(?:notengo|noexiste|nocuentaconcorreo)\b', # Frases comunes que indican falta de correo
    r'\bcorreoprueba\b',                 # Placeholder "correoprueba"
    r'ñ',                                # Caracter "ñ"
    r'^.*clienteizzi.*$',                # Correos que inicien con cliente izzi
    r'^(.*izzi.*)@.*$',
    r'^(1\.21|1212|1122|121|12.1|1\.2)', # Correos que jueguen con la secuencia de números 1 y 2
    r'^(618)',                           # Correos que jueguen con la secuencia de números 6
    r'^(sinusuario|demientras|notengo)', # Frases comunes que indican falta de correo
    r'^[._]',                            # Correos que comienzan con punto o guion bajo
    r'^[^a-zA-Z0-9]',                    # Correos que comienzan con caracteres no alfanuméricos
    r'^0+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',# Nuevas condiciones: solo ceros antes del dominio
    r'^1@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', # Nuevas condiciones: solo "1" antes del arroba
    r'^virtual@.*',                      # Solo elimina correos que comienzan con "virtual"
    r'^(notiene|notienecorreo|confirmaciones|provisional)@.*$', #Placeholders
    r'^(sincorreo|sinnombre|na|izzi.generico)@.*$', #Placeholders
    r'^facturas[-_]?izzi.*$',                       #facturas_izzi o facturasizzi
    r'^(clientenuevo|clienteizzi|izzi|aaa|om)@.*$', #Placeholders
    r'^(lacasadelbrujo|nohaycorreo|ninguno)@.*$',   #Placeholders
    r'^(manololo|sinmail|n|ngonzalez|generico)@.*$',#Placeholders
    r'^(atencionalcliente|izzigenerico|p-ngonzalezg|a.empresarial)@.*$',#Placeholders
    r'^(ventas\d+clientes|xxx|generico\d+)@.*$',    #Placeholders
    r'^(jesus|jose)@.*$|^sincorreo\d+@.*$',         #Nombres de tecnicos
    r'^(correo|123456|sin.correo|no.se.lo.sabe|wizzgenerico)@.*$',#Placeholders
    r'^(1234123|1.g|001|no.tengo|no.tiene|no|nocuenta)@.*$',#Placeholders
    r'^(abc|abc123|atc|sincontacto|iz.z.ot.el.e)@.*$', #Placeholders
    r'^(no_tienecorreo|no_se_me_ocurrio_otro|no.tiene_correo)@.*$', #Placeholders
    r'^(no_tiene_correo|no.aplica|no.maneja.correo)@.*$', #Placeholders
    r'^(noasignacorreo|nobrinda|nocorreo|nocorreoelectronico)@.*$', #Placeholders
    r'^(nocuenta[\w-]+|nodejo[\w-]+|noda[\w-]+|nodesea[\w-]+)@', #No cuenta
    r'([\w.-]*notiene[\w.-]*@|[\w.-]+notiene[\w.-]*)', #"nocuenta" al inicio del @ o después 
    r'([\w.-]*@notiene[\w.-]*)',                    #"nocuenta" después del @
    r'^123@',                                       #Empieza con 123@ 
    r'^[^@]*$',                                     #No contiene @
    r'\.$',                                         #Termina con un punto final
    r'@.*@',                                        #Múltiples @ no consecutivos
    r"^[a-zA-Z0-9]{1,2}@[\w.-]+\.[a-zA-Z]{2,}$",    #1 o 2 dígitos antes del @
    r'^z+@',                                        #correos que solo contengan z antes del @
                                    

]

**USO DE FUNCIÓN**

In [6]:
limpiar_correos(r'C:\Users\lgonzalezc\Documents\Trabajo\DB\Newsletter\DIC\TOÑO\base2.csv',
                r'C:\Users\lgonzalezc\Documents\Trabajo\DB\Newsletter\DIC\TOÑO\base2_limpia.csv',
                r'C:\Users\lgonzalezc\Documents\Trabajo\DB\Newsletter\DIC\TOÑO\base2_correos_eliminados.csv',
                wrong_words_regex)

SECCIÓN DE NULOS
Número de registros antes de eliminar valores nulos en 'EMAIL': 420089
Número de registros después de eliminar valores nulos en 'EMAIL': 420089

SECCIÓN DE REGISTROS CON FRECUENCIAS ALTAS
Número de correos con frecuencia mayor a 4: 135
Número de filas eliminadas debido a duplicados con frecuencia mayor a 4: 1546

SECCIÓN DE LLENADO DE NOMBRES VACÍOS
Cantidad de nombres llenados con 'ESTIMADO': 7105

SECCIÓN DE CORREOS ELIMINADOS POR REGEX
Número de correos eliminados por patrones no deseados: 1548

SECCIÓN DE REPEP
El número de registros en el DataFrame antes de eliminar REPEP:416995
El número de registros REPEP es de : 23900

El número de registros en el DataFrame después de eliminar REPEP: 393095
El número de registros en el DataFrame despúes de la limpieza es de: 393095


'Felicidades, has limpiado tu base con éxito'

In [5]:

df2 = pd.read_csv (r'C:\Users\lgonzalezc\Documents\Trabajo\DB\IZZI MOVIL\BD_IZZI_MOVIL_LIMPIA_1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lgonzalezc\\Documents\\Trabajo\\DB\\IZZI MOVIL\\BD_IZZI_MOVIL_LIMPIA_1.csv'

In [20]:
df2.dtypes

PERIODO                     int64
CLIENTE                     int64
ANTIGUEDAD_RESIDENCIAL      int64
SEGMENTO                   object
PLAY                       object
SG_PLAY                    object
PAQUETE                    object
CODIGO_RPT                  int64
RPT                        object
NOMBRES                    object
AP_PATERNO                 object
AP_MATERNO                 object
EMAIL                      object
TELEFONO_FIJO             float64
CELULAR                    object
FLG_REPEP                  object
TIPO                       object
dtype: object

In [21]:
# Contar la frecuencia de cada correo electrónico
email_counts = df2['EMAIL'].value_counts().reset_index()
email_counts.columns = ['email', 'total_values']
email_counts['total_values'] = email_counts['total_values'].astype('int')

# Filtrar los correos duplicados (frecuencia mayor a 1) usando query
duplicated_emails = email_counts.query('total_values > 1')

# Ordenar los duplicados por frecuencia
duplicated_emails_sorted = duplicated_emails.sort_values(by='email', ascending=False)

# Mostrar el resultado
print(duplicated_emails_sorted)

                              email  total_values
2717  zurc_mueb.juquila@hotmail.com             2
3436          zuleperez48@gmail.com             2
897           zoojitos@yahoo.com.mx             2
813             zogima@yahoo.com.mx             2
1426      zoet.nirvana6@hotmail.com             2
...                             ...           ...
3033          4sarzenia4s@gmail.com             2
3471          4423386279z@gmail.com             2
3754   1477991gcastillo06@gmail.com             2
1404           022.victor@gmail.com             2
3584            016sector@gmail.com             2

[4229 rows x 2 columns]


In [22]:
email_counts

,email,total_values
0,yesenialokota@gmail.com,4
1,ricosalazar3@hotmail.com,4
2,paradisomarlin@hotmail.com,4
3,financiamiento1999@gmail.com,4
4,him666_@hotmail.com,4
...,...,...
498734,mpepe2854@gmail.com,1
498735,oscarzaratem43@gmail.com,1
498736,em62251@gmail.com,1
498737,orlando.terrazas@hotmail.com,1


In [23]:
duplicated_emails_sorted.head(100).iloc[31:50]

,email,total_values
2746,yolivallet@gmail.com,2
3029,yolicruz1655@gmail.com,2
1282,yoli.escutia1401@gmail.com,2
4120,yolandahernandez@gmail.com,2
38,yolanda_gonzalez_lopez@hotmail.com,3
3522,yolamarquez@live.com,2
1452,yolaglez_1oct@hotmail.com,2
3251,yojan.palacios@gmail.com,2
801,yogagomez@gmail.com,2
39,yferro@toks.com.mx,3
